### Задача

Нарезать видео на кадры из которых будет формироваться датасет для обучения.

1. Необходимо нарезать видео из ФСК.
2. Необходимо нарезать видео из Самолета.

### Дата нарезки

05.02.2024. По дате определим, какие видео для добавления в датасет, отбирать в будущем.

### Mount

Подмонтируем папку из сервера к докер контейнеру Jupyter Lab.

In [ ]:
sudo mount --bind /mnt/4TBVideo/Video /var/lib/docker/volumes/jupyterhub-user-{git-username}/_data/samolet_data_video

In [ ]:
/var/lib/docker/volumes/jupyterhub-user-ignatovd/_data/csm_inner_ds/cv/classification/multi/decor/v0.1/data/datasets/original

### План нарезки

Обсудили с заказчиком, что необходимо сделать упор на улучшение работы модели на данных заказчика ФСК. На момент нарезки, было отснято 98 видео одного здания ФСК. Все видео были взяты в нарезку изображений в датасет. Последнее видео от 01.02.24.

В проекте Самолет проходит пилот на 4 зданиях. Проекты отличаются от тех, что были ранее в обучающей выборке. Хорошо их добавить для улучшения обобщающей способности модели. Чтобы успеть в сроки по времени и с учетом имеющихся ресурсов разметчиков, было отобрано несколько видео вручную для добавления в датасет. Отбор происходил по принципу, чтобы добавить кейсы которые модель не видела раньше. Названия видео представлены ниже, если запланируем в будущем добавить больше данных из этих проектов, мы по названию исключим видео из нарезки, чтобы исключить дубли. 

По расчётам сроков и доступному количеству разметчиков, всего разметить сможем 3300 изображений. Необходимо в данной итерации добавить следующие датасеты в необходимом количестве:

1. Водяные счётчики, 300 шт.
2. Двери МОП, 500 шт.
3. Изображения на котором изображено окно, помещение в черновой отделке без батареи, 500 шт.
4. Здание ФСК, 1500 шт.
5. Пилотные проекты Самолета, 500 шт.

**Водяные счетчики.**

Собрали в интернете изображения, потому что водяные счетчики в Самолете устанавливают при здаче квартиры заказчку, т.к. счетчики воруют. Поэтому на видео водяные счетчики присутствуют весьма редко.

**Двери МОП**

Ожидаем съемку, отдельно только дверей МОП. На видео объодчиков двери МОП встречаются редко.

**Пилотные проекты самолета.**

На данный момент ведется съемка на 4х объектах: Прибрежный парк (ПРП_3 оч_8.2 ж.д.), Пригород Лесное (ПЛ_6 оч_1 эт_8.1 ж.д.), Новое Внуково (НВ_2 оч_13 ж.д.), Остафьево есть в текущем датасете.

Dubai_SHOBA CONSTRUCTION_[Building 1] есть небольшое короткое видео офиса с готовой отделкой, возьмем так же немного изображений. 

In [37]:
pilot_videos = [
    # квартиры на этаже
    'QR_Московская обл._ПРИБРЕЖНЫЙ ПАРК_[ПРП_3 оч_8.2 ж.д.]_1_7_125_o5RIRB_04-02-24_decor.mp4',
    'Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_3_117_npHBS5_24-01-24_decor.mp4',
    'QR_Dubai_SHOBA CONSTRUCTION_[Building 1]_1_3_11_7CAeyk_12-12-23_decor.mp4',
    'QR_Московская обл._ПРИГОРОД ЛЕСНОЕ_[ПЛ_6 оч_1 эт_8.1 ж.д.]_1_2_127_IhppQE_11-12-23_decor.mp4',
    # вестибюль
    # Новое Внуково, есть только в финальной стадии
    'Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_1_117_zQp23A_18-11-23_decor.mp4',
    # Прибрежный парк есть только в черновой стадии
    'QR_Московская обл._ПРИБРЕЖНЫЙ ПАРК_[ПРП_3 оч_8.2 ж.д.]_1_1_125_Qq281y_29-10-23_decor.mp4',
    # Остафьево Пригород Лесное видео вестибюля отсутствует
]

**Изображение окна, помещения в черновой отделки без батареи.**

In [ ]:
radiator_videos = [
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_11_126_vH8Awc_07-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_10_126_ALZsWH_21-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_9_126_HloViF_21-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_4_126_RhA3jm_21-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_6_126_4a8o76_21-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_6_126_7nk4A9_07-10-23_decor.mp4'
]

### Import

In [36]:
import os
import numpy as np
import cv2
import time
from tqdm import tqdm
from pathlib import Path

### Config

In [23]:
PATH = '../data/outer/video/fsk/source_video_decor/'
PATH_SAVE = '../data/cutting/fsk/'
multiplier = 1000

### Cutting without select

In [ ]:
for root, dirs, files in os.walk(PATH, topdown=False):
    if len(files) != 0:
        print(f'Number of video: {len(files)}')
        bar = tqdm(files)
        for name in bar:
            bar.set_description(f'Video: {name}')
            if name.endswith(".mp4") or name.endswith(".MP4"):
                count = 0
                path = os.path.join(root, name)
                print(f'Path video: {path}')
                cap = cv2.VideoCapture(path)
                length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                print(f'Number of frames: {length}')

                while(cap.isOpened() and count < length):
                    count += 1
                    ret, frame = cap.read()
                    if ret:
                        if (count % multiplier == 0):
                            status = cv2.imwrite(PATH_SAVE+Path(name).stem+"_frame%d.jpg" % count, frame)
                            #print("Image written to file-system : ",count)

                cap.release()

Number of video: 98


Video: QR_Московская обл._БИТЦА_[7й корпус]_2_16_3_az3Cgz_26-01-24.mp4:   0%|          | 0/98 [00:00<?, ?it/s]

Path video: ../data/outer/video/fsk/source_video_decor/QR_Московская обл._БИТЦА_[7й корпус]_2_16_3_az3Cgz_26-01-24.mp4
Number of frames: 5358


Video: QR_Московская обл._БИТЦА_[7й корпус]_2_19_3_WKygOQ_26-01-24.mp4:   1%|          | 1/98 [01:51<3:00:40, 111.76s/it]

Path video: ../data/outer/video/fsk/source_video_decor/QR_Московская обл._БИТЦА_[7й корпус]_2_19_3_WKygOQ_26-01-24.mp4
Number of frames: 5848


...

Path video: ../data/outer/video/fsk/source_video_decor/QR_Московская обл._БИТЦА_[7й корпус]_1_4_3_4etLQu_25-01-24_decor.mp4
Number of frames: 4806


Video: QR_Московская обл._БИТЦА_[7й корпус]_1_17_3_it2WP0_26-01-24.mp4:  36%|███▌      | 35/98 [30:43<1:03:40, 60.65s/it]     

Path video: ../data/outer/video/fsk/source_video_decor/QR_Московская обл._БИТЦА_[7й корпус]_1_17_3_it2WP0_26-01-24.mp4
Number of frames: 3029


Video: QR_Московская обл._БИТЦА_[7й корпус]_4_18_3_HCA3C4_30-01-24.mp4:  37%|███▋      | 36/98 [31:28<57:43, 55.87s/it]  

Path video: ../data/outer/video/fsk/source_video_decor/QR_Московская обл._БИТЦА_[7й корпус]_4_18_3_HCA3C4_30-01-24.mp4
Number of frames: 3131


Необходимо отобрать 1500 кадров.

In [ ]:
# заархивировать в tar архив
def create_relative_tar(dir_to_compress, tar_filename):
    with tarfile.open(tar_filename, 'w') as tar:
        for root, _, files in os.walk(dir_to_compress):
            for file in files:
                full_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_path, dir_to_compress)
                arcname = os.path.join(os.path.basename(dir_to_compress), relative_path)
                tar.add(full_path, arcname=arcname)

In [134]:
k = 1500

In [135]:
import os
import cv2
import shutil
import random
import tarfile
from pathlib import Path
from tqdm import tqdm


dir_save = '../data/cutting/fsk_selected'

if os.path.exists(dir_save):
    shutil.rmtree(dir_save)
    
os.makedirs(dir_save, exist_ok=True)

path_imgs = [str(p) for p in Path(PATH_SAVE).rglob('*.jpg')]
num_imgs = len(path_imgs)

indxs = random.sample(range(num_imgs), k=k)

for indx in tqdm(indxs):
    path_img = path_imgs[indx]
    file_name = Path(path_img).stem
    img = cv2.imread(path_img)
    cv2.imwrite(f'{dir_save}/{file_name}.jpg', img)
    
create_relative_tar(dir_save, f'{dir_save}.tar')

100%|██████████| 1500/1500 [01:57<00:00, 12.74it/s]


### Cutting with select

Нарезаем только указанные видео.

### Radiator

In [52]:
selected_videos = [
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_11_126_vH8Awc_07-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_10_126_ALZsWH_21-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_9_126_HloViF_21-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_4_126_RhA3jm_21-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_6_126_4a8o76_21-10-23_decor.mp4',
    'QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_6_126_7nk4A9_07-10-23_decor.mp4'
]

In [53]:
PATH = '../data/outer/video/constructly/source_video/'
PATH_SAVE = '../data/cutting/radiator/'
multiplier = 15

In [54]:
for root, dirs, files in os.walk(PATH, topdown=False):
    if len(files) != 0:
        print(f'Number of video: {len(files)}')
        bar = tqdm(files)
        for name in bar:
            if name in selected_videos:
                bar.set_description(f'Video: {name}')
                if name.endswith(".mp4") or name.endswith(".MP4"):
                    count = 0
                    path = os.path.join(root, name)
                    print(f'Path video: {path}')
                    cap = cv2.VideoCapture(path)
                    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    print(f'Number of frames: {length}')

                    while(cap.isOpened() and count < length):
                        count += 1
                        ret, frame = cap.read()
                        if ret:
                            if (count % multiplier == 0):
                                status = cv2.imwrite(PATH_SAVE+Path(name).stem+"_frame%d.jpg" % count, frame)
                                #print("Image written to file-system : ",count)

                    cap.release()

Number of video: 473


Video: QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_10_126_ALZsWH_21-10-23_decor.mp4:   0%|          | 0/473 [00:00<?, ?it/s]

Path video: ../data/outer/video/constructly/source_video/QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_10_126_ALZsWH_21-10-23_decor.mp4
Number of frames: 24009


Video: QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_9_126_HloViF_21-10-23_decor.mp4:  18%|█▊        | 83/473 [01:00<04:43,  1.38it/s] 

Path video: ../data/outer/video/constructly/source_video/QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_9_126_HloViF_21-10-23_decor.mp4
Number of frames: 22743


...

Video: QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_11_126_vH8Awc_07-10-23_decor.mp4:  55%|█████▌    | 261/473 [03:24<03:03,  1.16it/s]

Path video: ../data/outer/video/constructly/source_video/QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_11_126_vH8Awc_07-10-23_decor.mp4
Number of frames: 21312


Video: QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_6_126_4a8o76_21-10-23_decor.mp4:  60%|██████    | 284/473 [04:14<03:29,  1.11s/it] 

Path video: ../data/outer/video/constructly/source_video/QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_6_126_4a8o76_21-10-23_decor.mp4
Number of frames: 19838


Video: QR_Москва_ОСТАФЬЕВО_[ОСТ_4 оч_13 ж.д.]_1_6_126_4a8o76_21-10-23_decor.mp4: 100%|██████████| 473/473 [04:59<00:00,  1.58it/s]


### Пилот Самолета

In [55]:
selected_videos = [
    # квартиры на этаже
    'QR_Московская обл._ПРИБРЕЖНЫЙ ПАРК_[ПРП_3 оч_8.2 ж.д.]_1_7_125_o5RIRB_04-02-24_decor.mp4',
    'Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_3_117_npHBS5_24-01-24_decor.mp4',
    'QR_Dubai_SHOBA CONSTRUCTION_[Building 1]_1_3_11_7CAeyk_12-12-23_decor.mp4',
    'QR_Московская обл._ПРИГОРОД ЛЕСНОЕ_[ПЛ_6 оч_1 эт_8.1 ж.д.]_1_2_127_IhppQE_11-12-23_decor.mp4',
    # вестибюль
    # Новое Внуково, есть только в финальной стадии
    'Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_1_117_zQp23A_18-11-23_decor.mp4',
    # Прибрежный парк есть только в черновой стадии
    'QR_Московская обл._ПРИБРЕЖНЫЙ ПАРК_[ПРП_3 оч_8.2 ж.д.]_1_1_125_Qq281y_29-10-23_decor.mp4',
    # Остафьево Пригород Лесное видео вестибюля отсутствует
]

In [56]:
PATH = '../data/outer/video/constructly/source_video/'
PATH_SAVE = '../data/cutting/samolet/'
multiplier = 50

In [57]:
for root, dirs, files in os.walk(PATH, topdown=False):
    if len(files) != 0:
        print(f'Number of video: {len(files)}')
        bar = tqdm(files)
        for name in bar:
            if name in selected_videos:
                bar.set_description(f'Video: {name}')
                if name.endswith(".mp4") or name.endswith(".MP4"):
                    count = 0
                    path = os.path.join(root, name)
                    print(f'Path video: {path}')
                    cap = cv2.VideoCapture(path)
                    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    print(f'Number of frames: {length}')

                    while(cap.isOpened() and count < length):
                        count += 1
                        ret, frame = cap.read()
                        if ret:
                            if (count % multiplier == 0):
                                status = cv2.imwrite(PATH_SAVE+Path(name).stem+"_frame%d.jpg" % count, frame)
                                #print("Image written to file-system : ",count)

                    cap.release()

Number of video: 473


Video: QR_Dubai_SHOBA CONSTRUCTION_[Building 1]_1_3_11_7CAeyk_12-12-23_decor.mp4:   0%|          | 0/473 [00:00<?, ?it/s]

Path video: ../data/outer/video/constructly/source_video/QR_Dubai_SHOBA CONSTRUCTION_[Building 1]_1_3_11_7CAeyk_12-12-23_decor.mp4
Number of frames: 533


Video: QR_Московская обл._ПРИБРЕЖНЫЙ ПАРК_[ПРП_3 оч_8.2 ж.д.]_1_1_125_Qq281y_29-10-23_decor.mp4:   3%|▎         | 16/473 [00:05<02:38,  2.89it/s]

Path video: ../data/outer/video/constructly/source_video/QR_Московская обл._ПРИБРЕЖНЫЙ ПАРК_[ПРП_3 оч_8.2 ж.д.]_1_1_125_Qq281y_29-10-23_decor.mp4
Number of frames: 954


...

Video: Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_1_117_zQp23A_18-11-23_decor.mp4:  39%|███▉      | 186/473 [04:41<07:32,  1.58s/it]               

Path video: ../data/outer/video/constructly/source_video/Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_1_117_zQp23A_18-11-23_decor.mp4
Number of frames: 1943


Video: Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_3_117_npHBS5_24-01-24_decor.mp4:  51%|█████     | 241/473 [05:05<04:40,  1.21s/it]

Path video: ../data/outer/video/constructly/source_video/Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_3_117_npHBS5_24-01-24_decor.mp4
Number of frames: 5340


Video: Hand_Москва_НОВОЕ ВНУКОВО_[НВ_2 оч_13 ж.д.]_1_3_117_npHBS5_24-01-24_decor.mp4: 100%|██████████| 473/473 [06:04<00:00,  1.30it/s]


Из нарезанных кадров отобрать 500 изображений без повторений.

In [130]:
# заархивировать в tar архив
def create_relative_tar(dir_to_compress, tar_filename):
    with tarfile.open(tar_filename, 'w') as tar:
        for root, _, files in os.walk(dir_to_compress):
            for file in files:
                full_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_path, dir_to_compress)
                arcname = os.path.join(os.path.basename(dir_to_compress), relative_path)
                tar.add(full_path, arcname=arcname)

In [132]:
import os
import cv2
import shutil
import random
import tarfile
from pathlib import Path
from tqdm import tqdm


dir_save = '../data/cutting/samolet_selected'

if os.path.exists(dir_save):
    shutil.rmtree(dir_save)
    
os.makedirs(dir_save, exist_ok=True)

path_imgs = [str(p) for p in Path(PATH_SAVE).rglob('*.jpg')]
num_imgs = len(path_imgs)

indxs = random.sample(range(num_imgs), k=500)

for indx in tqdm(indxs):
    path_img = path_imgs[indx]
    file_name = Path(path_img).stem
    img = cv2.imread(path_img)
    cv2.imwrite(f'{dir_save}/{file_name}.jpg', img)
    
create_relative_tar(dir_save, f'{dir_save}.tar')

100%|██████████| 500/500 [00:39<00:00, 12.78it/s]


In [ ]:
sudo cp /var/lib/docker/volumes/jupyterhub-user-ignatovd/_data/csm_inner_ds/cv/classification/multi/decor/v0.1/data/cutting/fsk_selected.tar /var/lib/docker/volumes/jupyterhub-user-meugeny/_data/for_cvat